In [2]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [37]:
import numpy as np
import pandas as pd

import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from ast import literal_eval
from print_time import print_time, print_msg
from df_addons import df_to_excel
from mts_paths import *

__import__("warnings").filterwarnings('ignore')

build_model = True

SEED = 2023
MY_DATA_PATH = WORK_PATH

In [38]:
file_logs = Path(r'D:\python-txt\mts\gini_male.logs')
if not file_logs.is_file():
    with open(file_logs, mode='a') as log:
        log.write('num;DS;als_factors;als_column;url_male_als_factors;'
                  'ROC_AUC;GINI;name_emb_factors;model_columns;'
                  'category_columns;comment\n')
    max_num = 0
else:
    df = pd.read_csv(file_logs, sep=';')
    if 'num' not in df.columns:
        df.insert(0, 'num', 0)
        df.num = df.index + 1
    if 'DS' not in df.columns:
        df.insert(1, 'DS', 'DS1')
        df.num = df.index + 1
    if 'comment' not in df.columns:
        df['comment'] = ''
    df.als_column = df.als_column.map(
        lambda x: literal_eval(x) if x[0] == '(' else x)
    df.als_column = df.als_column.map(
        lambda x: ", ".join(x) if isinstance(x, (list, tuple)) else x)
    df.num = df.index + 1
    max_num = df.num.max()

# for url_male_als_factors in range(150, 251, 10):
for als_factors in (97,):

    name_emb_factors = []
    # количество признаков
    als_factors = 97
    # als_factors = 100
    # als_factors = 104
    # als_factors = 150
    name_csv = f'url_usr_emb_factors_{als_factors}.csv'
    url_usr_emb_factors = MY_DATA_PATH.joinpath(name_csv.replace('.csv',
                                                                 '.feather'))
    name_emb_factors.append(url_usr_emb_factors)

    # количество признаков
    region_als_factors = 20
    name_csv = f'region_usr_emb_factors_{region_als_factors}.csv'
    region_usr_emb_factors = MY_DATA_PATH.joinpath(name_csv.replace('.csv',
                                                                    '.feather'))
    name_emb_factors.append(region_usr_emb_factors)

    # количество признаков
    city_als_factors = 30
    name_csv = f'city_usr_emb_factors_{city_als_factors}.csv'
    city_usr_emb_factors = MY_DATA_PATH.joinpath(name_csv.replace('.csv',
                                                                  '.feather'))
    # name_emb_factors.append(city_usr_emb_factors)

    # количество признаков
    # url_male_als_factors = 100
    url_male_als_factors = 200
    # male_prs_1=0.730036, male_user_prs_1=0.730056, male_avg_prs_1=0.729514
    als_columns_male = ('male_prs_1', 'male_user_prs_1', 'male_avg_prs_1')
    # als_columns_male = ('male_prs_1', 'male_user_prs_1')
    # als_columns_male = ('male_prs_1',)
    # als_columns_male = ('male_user_prs_1',)
    als_columns_male = ('male_avg_prs_1',)
    for als_column in als_columns_male:
        name_a = f'url_{als_column}_emb_factors_{url_male_als_factors}.feather'
        url_male_emb_factors = MY_DATA_PATH.joinpath(name_a)
        if not url_male_emb_factors.is_file():
            continue
        print(url_male_emb_factors)
        name_emb_factors.append(url_male_emb_factors)
    #

    targets = pd.read_feather(file_users)
    df = pd.read_feather(df_train_users)
    df = targets.merge(df, how='left', on='user_id')
    df.fillna(0, inplace=True)
    # удалить колонку index
    for col in ('index', 'url_host_count_distinct_y'):
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)

    print(df.columns)

    # df['is_male'].value_counts()

    all_model_columns = [
        'age', 'is_male', 'user_id',
        # 'region_name', 'city_name', 'cpe_id', 'firm_id',
        'request_cnt_sum',
        'url_host_count',
        'url_host_count_distinct',
        'different_city',
        'date_count',
        'city_name_count_distinct',
        'region_name_count_distinct',
        'date_count_distinct',
        'male_prs_1',
        # 'male_user_prs_1',  # с этим меньше
        # 'male_avg_1',  #
        'age_prs_1', 'age_prs_2', 'age_prs_3', 'age_prs_4', 'age_prs_5',
        'age_prs_6',
        'age_user_prs_1', 'age_user_prs_2', 'age_user_prs_3', 'age_user_prs_4',
        'age_user_prs_5', 'age_user_prs_6',
        # 'age_avg_1','age_avg_2', 'age_avg_3', 'age_avg_4', 'age_avg_5', 'age_avg_6',
        'pd_prs_0', 'pd_prs_1', 'pd_prs_2', 'pd_prs_3',
        'url_m_0_sum', 'url_m_1_sum',  # GINI по полу 0.724
        'url_u_0_sum', 'url_u_1_sum',  # GINI по полу 0.724
        # 'url_a_0_sum', 'url_a_1_sum', # GINI по полу 0.716
        'fm_r0_sum', 'fm_u0_sum',
        'fm_r1_sum', 'fm_u1_sum',
        'fame0user_sum', 'fame_user_sum'
    ]

    age_model_columns = ['url_g_1_sum', 'url_g_2_sum', 'url_g_3_sum',
                         'url_g_4_sum', 'url_g_5_sum', 'url_g_6_sum',
                         'url_p_1_sum', 'url_p_2_sum', 'url_p_3_sum',
                         'url_p_4_sum', 'url_p_5_sum', 'url_p_6_sum',
                         'fa_r1_sum', 'fa_r2_sum', 'fa_r3_sum',
                         'fa_r4_sum', 'fa_r5_sum', 'fa_r6_sum',
                         'fa_u1_sum', 'fa_u2_sum', 'fa_u3_sum',
                         'fa_u4_sum', 'fa_u5_sum', 'fa_u6_sum']

    # для предсказания пола - это плохо годится
    # all_model_columns.extend(age_model_columns)

    # колонки с категориями
    all_categories = (
        'cpe_id',
        'firm_id',
        # 'region_name',
        'city_name',
        'url_m',
        'url_u',
        'url_a',
    )

    learn_exclude = ['fame0user_sum', 'region_name', 'price_mean', ]

    model_columns = []
    for col in all_model_columns:
        if col in df.columns and col not in learn_exclude:
            if col not in model_columns:
                model_columns.append(col)

    # маркировка сайта  -1 - женский,  1 - мужской, 0 - нейтральный
    # df['url_m'] = np.sign(df.url_m_1_sum - df.url_m_0_sum)
    # df['url_u'] = np.sign(df.url_u_1_sum - df.url_u_0_sum)
    # df['url_a'] = np.sign(df.url_a_1_sum - df.url_a_0_sum)

    # формирование списка колонок с категориями и
    # добавление маркеров в категории если они присутствуют
    category_columns = []
    for col in all_categories:
        if col in df.columns and col not in learn_exclude:
            category_columns.append(col)
            if col not in model_columns:
                model_columns.append(col)

    df = df[model_columns]

    print('Обучаюсь на колонках:', model_columns)
    print('Категорийные колонки:', category_columns)
    print('Исключенные колонки:', learn_exclude)

    for col in category_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)

    train_columns = df.columns[2:]
    train_dtypes = df.dtypes[2:]

    # url_usr_emb_factors, region_usr_emb_factors, city_usr_emb_factors,
    # url_male_emb_factors
    # без ембендингов по городам GINI по полу 0.710
    # name_emb_factors = (url_usr_emb_factors, region_usr_emb_factors,
    #                     url_male_emb_factors)
    # name_emb_factors = (url_usr_emb_factors, region_usr_emb_factors)
    for emb_file in name_emb_factors:
        usr_emb = pd.read_feather(emb_file)
        df = df.merge(usr_emb, how='left', on='user_id')

    train = df.drop(['user_id', 'age', 'is_male'], axis=1)
    target = df['is_male']

    X_train, X_valid, y_train, y_valid = train_test_split(train, target,
                                                          test_size=0.25,
                                                          shuffle=True,
                                                          random_state=SEED,
                                                          # stratify=target
                                                          # stratify = 0.723963
                                                          )
    pool_train = Pool(data=X_train, label=y_train,
                      cat_features=category_columns)
    pool_valid = Pool(data=X_valid, label=y_valid,
                      cat_features=category_columns)

    clf = CatBoostClassifier(cat_features=category_columns,
                             auto_class_weights='Balanced',
                             loss_function='Logloss',
                             eval_metric='AUC:hints=skip_train~false',
                             # max_depth=8,
                             iterations=3000,  # попробовать столько итераций
                             # learning_rate=0.08,
                             early_stopping_rounds=50,
                             random_seed=SEED
                             )

D:\python-datasets\mts_context_data\url_male_avg_prs_1_emb_factors_200.feather
Index(['age', 'is_male', 'user_id', 'request_cnt_sum', 'url_host_count',
       'url_host_count_distinct', 'different_city', 'date_count',
       'region_name', 'city_name', 'city_name_count_distinct',
       'region_name_count_distinct', 'date_count_distinct', 'cpe_id',
       'firm_id', 'price_mean', 'male_prs_1', 'male_user_prs_1', 'male_avg_1',
       'age_prs_1', 'age_prs_2', 'age_prs_3', 'age_prs_4', 'age_prs_5',
       'age_prs_6', 'age_user_prs_1', 'age_user_prs_2', 'age_user_prs_3',
       'age_user_prs_4', 'age_user_prs_5', 'age_user_prs_6', 'age_avg_1',
       'age_avg_2', 'age_avg_3', 'age_avg_4', 'age_avg_5', 'age_avg_6',
       'pd_prs_0', 'pd_prs_1', 'pd_prs_2', 'pd_prs_3', 'url_m_0_sum',
       'url_m_1_sum', 'url_u_0_sum', 'url_u_1_sum', 'url_a_0_sum',
       'url_a_1_sum', 'fm_r0_sum', 'fm_u0_sum', 'fm_r1_sum', 'fm_u1_sum',
       'fame0user_sum', 'fame_user_sum', 'url_g_1_sum', 'url_g_2_su

In [41]:
train.isna().sum().sum()

0

In [29]:
%%time

start_time = print_msg('Запускаю классификатор...')

use_grid_search = True

if not use_grid_search:
    clf.fit(pool_train, eval_set=pool_valid,
            use_best_model=True, verbose=50)

    best_params = {'clf.tree_count_': clf.tree_count_, 
                   'clf.learning_rate': clf.get_all_params()['learning_rate']}
else:
    n_fold = 4
    grid_params = {
        'max_depth': [7],
        # 'max_depth': [6, 7, 8],
        # 'learning_rate': [0.01, 0.1, 1],
    }
    skf = StratifiedKFold(n_splits=n_fold, random_state=SEED, shuffle=True)
    # skf = KFold(n_splits=n_fold, random_state=SEED, shuffle=True)
    grid_search_result = clf.grid_search(grid_params, train, target,
                                         cv=skf,
                                         partition_random_seed=SEED,
                                         stratified=True,
                                         refit=True,
                                         plot=True)
    best_params = grid_search_result['params']


print_time(start_time)

print(best_params)

Запускаю классификатор...


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7074516	test: 0.7068007	best: 0.7068007 (0)	total: 360ms	remaining: 18m
1:	learn: 0.7245096	test: 0.7237786	best: 0.7237786 (1)	total: 705ms	remaining: 17m 37s
2:	learn: 0.7347084	test: 0.7336871	best: 0.7336871 (2)	total: 1.02s	remaining: 17m 2s
3:	learn: 0.7406401	test: 0.7401549	best: 0.7401549 (3)	total: 1.33s	remaining: 16m 34s
4:	learn: 0.7457261	test: 0.7457880	best: 0.7457880 (4)	total: 1.66s	remaining: 16m 33s
5:	learn: 0.7490748	test: 0.7490533	best: 0.7490533 (5)	total: 2.01s	remaining: 16m 41s
6:	learn: 0.7498198	test: 0.7498907	best: 0.7498907 (6)	total: 2.35s	remaining: 16m 43s
7:	learn: 0.7504136	test: 0.7507763	best: 0.7507763 (7)	total: 2.68s	remaining: 16m 40s
8:	learn: 0.7514486	test: 0.7519975	best: 0.7519975 (8)	total: 3.05s	remaining: 16m 54s
9:	learn: 0.7512369	test: 0.7515237	best: 0.7519975 (8)	total: 3.4s	remaining: 16m 55s
10:	learn: 0.7530705	test: 0.7534301	best: 0.7534301 (10)	total: 3.71s	remaining: 16m 47s
11:	learn: 0.7546841	test: 0.7550763

92:	learn: 0.8071952	test: 0.8049847	best: 0.8049847 (92)	total: 31.1s	remaining: 16m 11s
93:	learn: 0.8075224	test: 0.8053015	best: 0.8053015 (93)	total: 31.4s	remaining: 16m 10s
94:	learn: 0.8076797	test: 0.8054826	best: 0.8054826 (94)	total: 31.7s	remaining: 16m 10s
95:	learn: 0.8079573	test: 0.8057323	best: 0.8057323 (95)	total: 32s	remaining: 16m 9s
96:	learn: 0.8083495	test: 0.8061130	best: 0.8061130 (96)	total: 32.4s	remaining: 16m 9s
97:	learn: 0.8084860	test: 0.8062057	best: 0.8062057 (97)	total: 32.7s	remaining: 16m 9s
98:	learn: 0.8087258	test: 0.8063810	best: 0.8063810 (98)	total: 33s	remaining: 16m 7s
99:	learn: 0.8090206	test: 0.8066452	best: 0.8066452 (99)	total: 33.3s	remaining: 16m 7s
100:	learn: 0.8092763	test: 0.8069323	best: 0.8069323 (100)	total: 33.7s	remaining: 16m 7s
101:	learn: 0.8095921	test: 0.8072207	best: 0.8072207 (101)	total: 34.1s	remaining: 16m 8s
102:	learn: 0.8096645	test: 0.8072351	best: 0.8072351 (102)	total: 34.4s	remaining: 16m 8s
103:	learn: 0.81

183:	learn: 0.8247605	test: 0.8205077	best: 0.8205077 (183)	total: 1m 1s	remaining: 15m 37s
184:	learn: 0.8249319	test: 0.8206620	best: 0.8206620 (184)	total: 1m 1s	remaining: 15m 37s
185:	learn: 0.8250546	test: 0.8207542	best: 0.8207542 (185)	total: 1m 1s	remaining: 15m 37s
186:	learn: 0.8251817	test: 0.8208516	best: 0.8208516 (186)	total: 1m 2s	remaining: 15m 37s
187:	learn: 0.8253131	test: 0.8209512	best: 0.8209512 (187)	total: 1m 2s	remaining: 15m 36s
188:	learn: 0.8254595	test: 0.8210665	best: 0.8210665 (188)	total: 1m 2s	remaining: 15m 36s
189:	learn: 0.8255550	test: 0.8211213	best: 0.8211213 (189)	total: 1m 3s	remaining: 15m 35s
190:	learn: 0.8256955	test: 0.8212384	best: 0.8212384 (190)	total: 1m 3s	remaining: 15m 35s
191:	learn: 0.8258167	test: 0.8213299	best: 0.8213299 (191)	total: 1m 3s	remaining: 15m 34s
192:	learn: 0.8259519	test: 0.8214447	best: 0.8214447 (192)	total: 1m 4s	remaining: 15m 34s
193:	learn: 0.8260617	test: 0.8215577	best: 0.8215577 (193)	total: 1m 4s	remaini

...

2631:	learn: 0.9129193	test: 0.8596296	best: 0.8596296 (2631)	total: 13m 50s	remaining: 1m 56s
2632:	learn: 0.9129308	test: 0.8596328	best: 0.8596328 (2632)	total: 13m 50s	remaining: 1m 55s
2633:	learn: 0.9129500	test: 0.8596341	best: 0.8596341 (2633)	total: 13m 50s	remaining: 1m 55s
2634:	learn: 0.9129630	test: 0.8596358	best: 0.8596358 (2634)	total: 13m 51s	remaining: 1m 55s
2635:	learn: 0.9129831	test: 0.8596371	best: 0.8596371 (2635)	total: 13m 51s	remaining: 1m 54s
2636:	learn: 0.9129995	test: 0.8596366	best: 0.8596371 (2635)	total: 13m 51s	remaining: 1m 54s
2637:	learn: 0.9130239	test: 0.8596506	best: 0.8596506 (2637)	total: 13m 52s	remaining: 1m 54s
2638:	learn: 0.9130504	test: 0.8596633	best: 0.8596633 (2638)	total: 13m 52s	remaining: 1m 53s
2639:	learn: 0.9130707	test: 0.8596714	best: 0.8596714 (2639)	total: 13m 52s	remaining: 1m 53s
2640:	learn: 0.9130884	test: 0.8596703	best: 0.8596714 (2639)	total: 13m 53s	remaining: 1m 53s
2641:	learn: 0.9131100	test: 0.8596724	best: 0.859

2718:	learn: 0.9147135	test: 0.8597966	best: 0.8597974 (2717)	total: 14m 17s	remaining: 1m 28s
2719:	learn: 0.9147395	test: 0.8598003	best: 0.8598003 (2719)	total: 14m 17s	remaining: 1m 28s
2720:	learn: 0.9147651	test: 0.8598013	best: 0.8598013 (2720)	total: 14m 17s	remaining: 1m 27s
2721:	learn: 0.9147902	test: 0.8598029	best: 0.8598029 (2721)	total: 14m 18s	remaining: 1m 27s
2722:	learn: 0.9148161	test: 0.8598076	best: 0.8598076 (2722)	total: 14m 18s	remaining: 1m 27s
2723:	learn: 0.9148357	test: 0.8598086	best: 0.8598086 (2723)	total: 14m 18s	remaining: 1m 27s
2724:	learn: 0.9148582	test: 0.8598061	best: 0.8598086 (2723)	total: 14m 19s	remaining: 1m 26s
2725:	learn: 0.9148735	test: 0.8598065	best: 0.8598086 (2723)	total: 14m 19s	remaining: 1m 26s
2726:	learn: 0.9148943	test: 0.8598073	best: 0.8598086 (2723)	total: 14m 19s	remaining: 1m 26s
2727:	learn: 0.9149089	test: 0.8598087	best: 0.8598087 (2727)	total: 14m 20s	remaining: 1m 25s
2728:	learn: 0.9149327	test: 0.8598116	best: 0.859

2805:	learn: 0.9164563	test: 0.8600034	best: 0.8600034 (2805)	total: 14m 44s	remaining: 1m 1s
2806:	learn: 0.9164782	test: 0.8599981	best: 0.8600034 (2805)	total: 14m 44s	remaining: 1m
2807:	learn: 0.9164957	test: 0.8599968	best: 0.8600034 (2805)	total: 14m 45s	remaining: 1m
2808:	learn: 0.9165137	test: 0.8599986	best: 0.8600034 (2805)	total: 14m 45s	remaining: 1m
2809:	learn: 0.9165349	test: 0.8600048	best: 0.8600048 (2809)	total: 14m 45s	remaining: 59.9s
2810:	learn: 0.9165509	test: 0.8600042	best: 0.8600048 (2809)	total: 14m 46s	remaining: 59.6s
2811:	learn: 0.9165776	test: 0.8600101	best: 0.8600101 (2811)	total: 14m 46s	remaining: 59.3s
2812:	learn: 0.9166019	test: 0.8600208	best: 0.8600208 (2812)	total: 14m 46s	remaining: 58.9s
2813:	learn: 0.9166240	test: 0.8600197	best: 0.8600208 (2812)	total: 14m 46s	remaining: 58.6s
2814:	learn: 0.9166432	test: 0.8600198	best: 0.8600208 (2812)	total: 14m 47s	remaining: 58.3s
2815:	learn: 0.9166642	test: 0.8600228	best: 0.8600228 (2815)	total: 

2893:	learn: 0.9181741	test: 0.8601344	best: 0.8601360 (2889)	total: 15m 11s	remaining: 33.4s
2894:	learn: 0.9181968	test: 0.8601280	best: 0.8601360 (2889)	total: 15m 12s	remaining: 33.1s
2895:	learn: 0.9182226	test: 0.8601293	best: 0.8601360 (2889)	total: 15m 12s	remaining: 32.8s
2896:	learn: 0.9182479	test: 0.8601441	best: 0.8601441 (2896)	total: 15m 12s	remaining: 32.4s
2897:	learn: 0.9182635	test: 0.8601450	best: 0.8601450 (2897)	total: 15m 12s	remaining: 32.1s
2898:	learn: 0.9182857	test: 0.8601511	best: 0.8601511 (2898)	total: 15m 13s	remaining: 31.8s
2899:	learn: 0.9183076	test: 0.8601546	best: 0.8601546 (2899)	total: 15m 13s	remaining: 31.5s
2900:	learn: 0.9183290	test: 0.8601589	best: 0.8601589 (2900)	total: 15m 13s	remaining: 31.2s
2901:	learn: 0.9183490	test: 0.8601615	best: 0.8601615 (2901)	total: 15m 14s	remaining: 30.9s
2902:	learn: 0.9183686	test: 0.8601666	best: 0.8601666 (2902)	total: 15m 14s	remaining: 30.6s
2903:	learn: 0.9183843	test: 0.8601690	best: 0.8601690 (2903

2981:	learn: 0.9199129	test: 0.8603018	best: 0.8603018 (2981)	total: 15m 39s	remaining: 5.67s
2982:	learn: 0.9199261	test: 0.8603087	best: 0.8603087 (2982)	total: 15m 39s	remaining: 5.35s
2983:	learn: 0.9199455	test: 0.8603096	best: 0.8603096 (2983)	total: 15m 39s	remaining: 5.04s
2984:	learn: 0.9199654	test: 0.8603095	best: 0.8603096 (2983)	total: 15m 40s	remaining: 4.72s
2985:	learn: 0.9199843	test: 0.8603085	best: 0.8603096 (2983)	total: 15m 40s	remaining: 4.41s
2986:	learn: 0.9199984	test: 0.8603088	best: 0.8603096 (2983)	total: 15m 40s	remaining: 4.09s
2987:	learn: 0.9200195	test: 0.8603098	best: 0.8603098 (2987)	total: 15m 41s	remaining: 3.78s
2988:	learn: 0.9200394	test: 0.8603088	best: 0.8603098 (2987)	total: 15m 41s	remaining: 3.46s
2989:	learn: 0.9200627	test: 0.8603123	best: 0.8603123 (2989)	total: 15m 41s	remaining: 3.15s
2990:	learn: 0.9200834	test: 0.8603134	best: 0.8603134 (2990)	total: 15m 41s	remaining: 2.83s
2991:	learn: 0.9201078	test: 0.8603110	best: 0.8603134 (2990

In [34]:
clf.get_all_params()['learning_rate']

0.04065300151705742

In [35]:
clf.tree_count_

3000

In [31]:
roc_auc = m.roc_auc_score(y_valid, clf.predict_proba(X_valid)[:, 1])
print(f'ROC_AUC = {roc_auc:.6f}, GINI по полу {2 * roc_auc - 1:.6f}')

# MAX GINI по полу 0.731672
# iterations=3000 --> 0.732522
# iterations=3333, max_depth=9 --> 0.732557

comment = {'clf.tree_count_': clf.tree_count_}
comment.update(clf.get_params())

max_num += 1
with open(file_logs, mode='a') as log:
    log.write(f'{max_num};DS2;{als_factors};'
              f'{", ".join(als_columns_male)};{url_male_als_factors};'
              f'{roc_auc:.6f};{2 * roc_auc - 1:.6f};'
              f'{[file.name for file in name_emb_factors]};'
              f'{model_columns};{category_columns};{comment}\n')

ROC_AUC = 0.924328, GINI по полу 0.848656


In [32]:
if build_model:
    model = clf
    submit_prefix = 'cb_'
    submit_csv = f'{submit_prefix}submit_male_{max_num:03}.csv'
    file_submit_csv = PREDICTIONS_DIR.joinpath(submit_csv)

    test = pd.read_feather(df_test_users)
    test = test[train_columns]

    # установить тип данных как в тренировочном ДФ
    for col in test.columns:
        test[col] = test[col].astype(train_dtypes[col])

    #  добавление эмбендингов
    for emb_file in name_emb_factors:
        usr_emb = pd.read_feather(emb_file)
        test = test.merge(usr_emb, how='left', on='user_id')

    predictions = model.predict_proba(test.drop('user_id', axis=1))

    test['is_male'] = predictions[:, 1]

    submit_columns = ['user_id', 'is_male']
    submit = test[submit_columns]
    submit.to_csv(file_submit_csv, index=False)

    roc_auc = m.roc_auc_score(target, model.predict_proba(train)[:, 1])
    print(f'ROC_AUC на всём train = {roc_auc:.6f}, '
          f'GINI по полу {2 * roc_auc - 1:.6f}')

ROC_AUC на всём train = 0.923489, GINI по полу 0.846978


In [36]:
if build_model:
    model = CatBoostClassifier(cat_features=category_columns,
                               auto_class_weights='Balanced',
                               loss_function='Logloss',
                               eval_metric='AUC:hints=skip_train~false',
                               # max_depth=8,
                               # попробовать столько итераций
                               iterations=int(clf.tree_count_ * 1.1),
                               # learning_rate=0.01,
                               early_stopping_rounds=50,
                               random_seed=SEED,
                               )

    model.fit(train, target, verbose=50)

    submit_prefix = 'cb_'
    submit_csv = f'{submit_prefix}submit_male_{max_num:03}.csv'
    file_submit_csv = PREDICTIONS_DIR.joinpath(submit_csv)

    test = pd.read_feather(df_test_users)
    test = test[train_columns]

    # установить тип данных как в тренировочном ДФ
    for col in test.columns:
        test[col] = test[col].astype(train_dtypes[col])

    #  добавление эмбендингов
    for emb_file in name_emb_factors:
        usr_emb = pd.read_feather(emb_file)
        test = test.merge(usr_emb, how='left', on='user_id')

    predictions = model.predict_proba(test.drop('user_id', axis=1))

    test['is_male'] = predictions[:, 1]

    submit_columns = ['user_id', 'is_male']
    submit = test[submit_columns]
    submit.to_csv(file_submit_csv, index=False)

    roc_auc = m.roc_auc_score(target, model.predict_proba(train)[:, 1])
    print(f'ROC_AUC на всём train = {roc_auc:.6f}, '
          f'GINI по полу {2 * roc_auc - 1:.6f}')

Learning rate set to 0.037251
0:	learn: 0.6958469	total: 334ms	remaining: 18m 23s
50:	learn: 0.7883540	total: 15.9s	remaining: 16m 55s
100:	learn: 0.8085735	total: 31.5s	remaining: 16m 36s
150:	learn: 0.8189968	total: 47.7s	remaining: 16m 34s
200:	learn: 0.8263678	total: 1m 4s	remaining: 16m 32s
250:	learn: 0.8313108	total: 1m 21s	remaining: 16m 30s
300:	learn: 0.8352945	total: 1m 37s	remaining: 16m 13s
350:	learn: 0.8386881	total: 1m 53s	remaining: 15m 50s
400:	learn: 0.8417766	total: 2m 9s	remaining: 15m 36s
450:	learn: 0.8445678	total: 2m 26s	remaining: 15m 22s
500:	learn: 0.8469634	total: 2m 41s	remaining: 15m 2s
550:	learn: 0.8491977	total: 2m 57s	remaining: 14m 44s
600:	learn: 0.8512574	total: 3m 12s	remaining: 14m 23s
650:	learn: 0.8530601	total: 3m 27s	remaining: 14m 5s
700:	learn: 0.8547351	total: 3m 42s	remaining: 13m 46s
750:	learn: 0.8563226	total: 3m 58s	remaining: 13m 31s
800:	learn: 0.8578936	total: 4m 20s	remaining: 13m 31s
850:	learn: 0.8593432	total: 4m 37s	remaining:

In [17]:
comment

{'clf.tree_count_': 2246,
 'iterations': 3000,
 'loss_function': 'Logloss',
 'random_seed': 2023,
 'auto_class_weights': 'Balanced',
 'eval_metric': 'AUC:hints=skip_train~false',
 'early_stopping_rounds': 50,
 'cat_features': ['cpe_id', 'firm_id', 'city_name']}

In [ ]:
FileLink(file_logs)

## Это добавить в тренировку по возрасту

In [ ]:
# обучение на всей тренировочной выборке
clf.fit(train, target, verbose=100, cat_features=category_columns)

f1w = m.f1_score(target, clf.predict(train), average='weighted')
print(f'Weighted F1-score = {f1w:.6f}')
print(m.classification_report(target, clf.predict(train),
                              target_names=['19-25', '25-34', '35-44', '45-54', '55-65', '65+']))

file_submit = WORK_PATH.joinpath('submission.feather')
submit = pd.read_feather(file_submit)

test = pd.read_feather(df_test_users)
test = test[train_columns]

# установить тип данных как в тренировочном ДФ        
for col in test.columns:
    test[col] = test[col].astype(train_dtypes[col])  

#  добавление эмбендингов
for emb_file in name_emb_factors:
    usr_emb = pd.read_feather(emb_file)
    test = test.merge(usr_emb, how='left', on='user_id')    

test['age'] = clf.predict(test)

submit_prefix = 'cb_'
submit_csv = f'{submit_prefix}submit_age_{max_num:03}.csv'
file_submit_csv = PREDICTIONS_DIR.joinpath(submit_csv)

submit_columns = ['user_id', 'age']
submit = test[submit_columns]
submit.to_csv(file_submit_csv, index=False)

FileLink(file_submit)

In [ ]:
Sure, here are the main stages of binary classification using CatBoostClassifier with cross-validation and hyperparameter tuning using grid search:

Import the necessary libraries and load the data:
python
Copy code
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

train = pd.read_csv('train.csv')
target = pd.read_csv('target.csv')
test = pd.read_csv('test.csv')
Split the data into training and validation sets:
python
Copy code
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train, target, test_size=0.2, random_state=42)
Define the parameter grid for grid search:
python
Copy code
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1]
}
Create an instance of CatBoostClassifier:
python
Copy code
clf = CatBoostClassifier(iterations=1000, loss_function='Logloss', verbose=False)
Create an instance of GridSearchCV and fit the training data:
python
Copy code
grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train, cat_features=[0,1,2,3,4])
Evaluate the best hyperparameters and the model performance on the validation set:
python
Copy code
print('Best parameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)
y_pred = grid_search.predict(X_valid)
print('Validation set score:', clf.score(X_valid, y_valid))
Train the final model on the entire training set with the best hyperparameters:
python
Copy code
final_clf = CatBoostClassifier(iterations=1000, loss_function='Logloss', verbose=False, **grid_search.best_params_)
final_clf.fit(train, target, cat_features=[0,1,2,3,4])
Make predictions on the test set using the final model:
python
Copy code
test_pred = final_clf.predict(test)
That's it! This is a basic example of how to perform binary classification using CatBoostClassifier with cross-validation and hyperparameter tuning using grid search. Note that you may need to modify some parts of the code to fit your specific use case.

Overfitting occurs when the model is too complex and captures noise or idiosyncrasies of the training data, leading to poor generalization performance on new data. To minimize overfitting, we need to ensure that the model is not too complex and that it generalizes well to new data. Here are some of the most popular hyperparameters that can be tweaked to minimize overfitting:

max_depth: Controls the maximum depth of the tree. A lower value will result in a shallower tree, which is less likely to overfit.
n_estimators: The number of trees in the ensemble. A higher value will generally reduce overfitting, but at the cost of increased computational complexity.
subsample: The fraction of samples to use for each tree. Using a value less than 1.0 will result in subsampling the data, which can reduce overfitting.
colsample_bylevel: The fraction of features to use for each tree level. Using a value less than 1.0 will result in subsampling the features, which can reduce overfitting.
l2_leaf_reg: L2 regularization coefficient. A higher value will increase the strength of the L2 regularization penalty, which can help reduce overfitting.
It's important to note that the above hyperparameters may have trade-offs with other aspects of model performance, such as bias and computational complexity. Therefore, it's important to carefully tune these hyperparameters using techniques like cross-validation and monitoring performance on a validation set to find the best balance between overfitting and underfitting.

In [33]:
cpe_models = pd.read_feather(file_cpe_models)

In [34]:
cpe_models.isna().sum()

cpe_manufacturer_name    0
cpe_model_name           0
cpe_id                   0
firm_id                  0
price_mean               0
word                     0
dtype: int64

In [35]:
cpe_models[cpe_models.price_mean.isna()]

,cpe_manufacturer_name,cpe_model_name,cpe_id,firm_id,price_mean,word


In [36]:
cpe_models[cpe_models.cpe_model_name.str.startswith('Blade')]

,cpe_manufacturer_name,cpe_model_name,cpe_id,firm_id,price_mean,word
588,ZTE,Blade 20 Smart,588,36,12437.457120,blade
589,ZTE,Blade A3 2019,589,36,3717.626715,blade
590,ZTE,Blade A3 2020RU,590,36,5973.185385,blade
591,ZTE,Blade A5,591,36,7717.590737,blade
592,ZTE,Blade A6,592,36,8260.617384,blade
593,ZTE,Blade A6 Lite,593,36,6920.000000,blade
594,ZTE,Blade A610,594,36,7213.328229,blade
595,ZTE,Blade A7,595,36,7213.328229,blade
596,ZTE,Blade A7s,596,36,7213.328229,blade
597,ZTE,Blade L210RU,597,36,5286.898551,blade


In [21]:
cpe_models['word'] = cpe_models.cpe_model_name.map(lambda x: x.lower().split()[0] if ' ' in x else '')

In [15]:
cpe_models.price_mean = cpe_models.groupby(['firm_id', 'word'])['price_mean']\
    .transform(lambda x: x.fillna(x.mean()))

In [24]:
cpe_models[cpe_models.cpe_model_name.str.startswith('Blade')]

,cpe_manufacturer_name,cpe_model_name,cpe_id,firm_id,price_mean,word
588,ZTE,Blade 20 Smart,588,36,12437.457120,blade
589,ZTE,Blade A3 2019,589,36,3717.626715,blade
590,ZTE,Blade A3 2020RU,590,36,5973.185385,blade
591,ZTE,Blade A5,591,36,7717.590737,blade
592,ZTE,Blade A6,592,36,8260.617384,blade
593,ZTE,Blade A6 Lite,593,36,6920.000000,blade
594,ZTE,Blade A610,594,36,7213.328229,blade
595,ZTE,Blade A7,595,36,7213.328229,blade
596,ZTE,Blade A7s,596,36,7213.328229,blade
597,ZTE,Blade L210RU,597,36,5286.898551,blade


In [30]:
cpe_models.loc[cpe_models.price_mean.isnull(), 'price_mean'] = cpe_models.groupby(['firm_id', 'word']).price_mean.transform('mean')
cpe_models.loc[cpe_models.price_mean.isnull(), 'price_mean'] = cpe_models.groupby(['firm_id']).price_mean.transform('mean')
cpe_models.loc[cpe_models.price_mean.isnull(), 'price_mean'] = cpe_models.price_mean.mean()